### BB84 code

##### imports

In [260]:
from qiskit import QuantumCircuit , transpile
from qiskit_aer import AerSimulator
import random

In [261]:
simulator=AerSimulator()

In [262]:
key_length = 20

#### Valid Bases

In [263]:
alice_bits = [ random . randint (0 , 1) for _ in range ( key_length ) ]
alice_bases = [ random . choice ([ 'X', 'Z']) for _ in range ( key_length ) ]
bob_bases = [ random . choice ([ 'X','Z']) for _ in range ( key_length ) ]

### Eve

In [264]:
eve_intercepted = [ random . choice ([ True , False]) for _ in range ( key_length )]

In [265]:
alice_qubits = []
for i in range ( key_length ) :
    qc = QuantumCircuit (1 , 1)
   # Prepare the qubit in |0 > or |1 > based on alice_bits
    if alice_bits [ i ] == 1:
         qc.x (0)

        # Apply basis (X for Hadamard , Z for no operation )
    if alice_bases [ i ] == 'X':
         qc.h (0)

    alice_qubits.append(qc)

#### Intercepted qubits

In [266]:
eve_results = []
for i , qc in enumerate ( alice_qubits ) :
    if eve_intercepted [ i ]:
        if random.choice(['X','Z']) =='Z':
            qc.h(0) # Measure in X basis
        qc.measure (0 , 0)
        transpiled_qc = transpile ( qc , simulator )
        result = simulator . run ( transpiled_qc ) . result ()
        counts = result . get_counts ()
if counts . get ('0', 0) == counts . get ('1', 0) :
    eve_results . append ( random . choice ([0 , 1]) )
else :
    eve_results . append ( int ( max ( counts , key = counts . get ) ) )

In [267]:
bob_results = []
for i in range ( key_length ) :
    # Create Bob ’s circuit with Alice ’s prepared qubit
    qc = alice_qubits [ i ]. copy ()
    if bob_bases [ i ] == 'X':
        qc . h (0)
    qc . measure (0 , 0)

    transpiled_qc = transpile ( qc , simulator )
    result = simulator . run ( transpiled_qc ) . result ()
    counts = result.get_counts()
    if counts . get ('0', 0) == counts . get ('1', 0) :
        bob_results . append ( random . choice ([0 , 1]) )
    else :
        bob_results . append ( int ( max ( counts , key = counts . get ) ) )


In [268]:
shared_key = []
eve_successful_intercepts = 0
total_matching_bases = 0

In [269]:
for i in range ( key_length) :
    print(len(bob_bases))
    print(len(bob_results))
    if alice_bases [ i ] == bob_bases [ i ]: # Keep only if bases match
        total_matching_bases += 1
        if alice_bits [ i ] == bob_results [ i ]: # Successful key bit
            shared_key . append ( alice_bits [ i ])
            if eve_intercepted [ i ]:
                eve_successful_intercepts += 1

20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20


In [270]:
if total_matching_bases > 0:
    eve_success_rate = eve_successful_intercepts / total_matching_bases
else :  
    eve_success_rate = 0 # Avoid division by zero if no matching bases
 # Display results
print (" Alice ’s Bits : ", alice_bits )
print (" Alice ’s Bases : ", alice_bases )
print (" Eve Intercepts : ", eve_intercepted )
print (" Bob ’s Bases : ", bob_bases )
print (" Bob ’s Results : ", bob_results )
print (" Shared Key : ", [ int ( bit ) for bit in shared_key])
print ( f" Eve ’s Success Rate : { eve_success_rate * 100}%")

 Alice ’s Bits :  [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1]
 Alice ’s Bases :  ['Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'Z']
 Eve Intercepts :  [False, False, True, False, True, False, False, False, False, True, True, True, True, True, False, False, True, False, False, False]
 Bob ’s Bases :  ['X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z']
 Bob ’s Results :  [1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
 Shared Key :  [1, 0, 1, 1, 0, 1, 1, 0, 1, 1]
 Eve ’s Success Rate : 23.076923076923077%


In [271]:
sample_size = len(shared_key)//5 +1 
print(sample_size)
sample_indices = random.sample(range(len(shared_key)), sample_size)

print(sample_indices)
errors = 0
for index in sample_indices:
    if shared_key[index] != bob_results[index]:
        errors += 1 
error_rate = errors / sample_size

if error_rate > 0.1:#threshold for security
    print("High error rate detected. Possible eavesdropping. Key discarded.")
else:
    print("Key is likely secure. Proceeding with the shared key.")


3
[2, 7, 4]
High error rate detected. Possible eavesdropping. Key discarded.


In [272]:

eve_intercepted = [ random . choice ([ True , False ]) for _ in range ( key_length )]